# CSC495 - DataMining - HW#4 - Rinty Chowdhury

In [66]:
import pandas as pd
import csv
import re
import string
import nltk
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

### Reading data and taking sample data from the original data

In [67]:
csv.field_size_limit(1000000000)
data = pd.read_csv('./1600000.processed.noemoticon.csv', names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                  encoding="iso-8859-1")

In [68]:
neg_data = data.loc[data['polarity'] == 0]
pos_data = data.loc[data['polarity'] == 4]

In [69]:
neg_data_sample = neg_data.sample(n = 16000)
pos_data_sample = pos_data.sample(n = 16000)

In [70]:
frames = [neg_data_sample, pos_data_sample]
tweet = pd.concat(frames)

### Preprocessing the data by removing urls, stopwords, tokenization, stemming, etc.

In [ ]:
def remove_url(text):
    tweet_out = re.sub(r'@[A-Za-z0-9]+','',text)
    tweet_new = re.sub(r'http://[A-Za-z0-9./]+','',tweet_out)
    return tweet_new
tweet['noUrl'] = tweet['text'].apply(lambda x: remove_url(x))
def remove_punc(text):
    text_out = "".join([char for char in text if char not in string.punctuation])
    return text_out
tweet['nopunc'] = tweet['noUrl'].apply(lambda x: remove_punc(x))
def tokanization(text):
    tokens = re.split('\W+', text)
    return tokens
tweet['tokens'] = tweet['nopunc'].apply(lambda x: tokanization(x.lower()))
ps = nltk.PorterStemmer()
def stemming(text):
    out_text = [ps.stem(word) for word in text]
    return out_text
tweet['stemdata'] = tweet['tokens'].apply(lambda x: stemming(x))
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text_out = [word for word in text if word not in stopword]
    return text_out
tweet['tweet2'] = tweet['stemdata'].apply(lambda x: remove_stopwords(x))
flat_list = list(itertools.chain.from_iterable(tweet['tweet2']))
fd = nltk.FreqDist(flat_list)
word_toKeep = list(filter(lambda x: 2000>x[1]>3, fd.items()))
word_list_toKeep = [item[0] for item in word_toKeep]
def remove_nonfreq(text):
    text_out = [word for word in text if word in word_list_toKeep]
    return text_out
tweet['tweet3'] = tweet['tweet2'].apply(lambda x: remove_nonfreq(x))
uniText = set(list(itertools.chain.from_iterable(tweet['tweet3'])))
def join_tokens(tokens):
    document = " ".join([word for word in tokens if not word.isdigit()])
    return document
tweet['thirdDoc'] = tweet['tweet3'].apply(lambda x: join_tokens(x))

In [72]:
cv = CountVectorizer(uniText)
countvec = cv.fit_transform(tweet['thirdDoc'])
countvecDF = pd.DataFrame(countvec.toarray())
countvecDF.columns = cv.get_feature_names()
X = countvecDF
y = tweet['polarity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

### Logistic Regression Classifier

In [73]:
LogisticRegression = LogisticRegression()
loreg = LogisticRegression.fit(X_train, y_train)
y_pred = LogisticRegression.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy: " , metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.72      0.74      3196
           4       0.73      0.76      0.75      3204

    accuracy                           0.74      6400
   macro avg       0.74      0.74      0.74      6400
weighted avg       0.74      0.74      0.74      6400

Accuracy:  0.74359375


## Precision score: 0.74
## Recall score: 0.74
## F-1 score: 0.74

### Precision score of negative tweet is 0.75 and positive tweet is 0.73. It means the model is 75% correct for negative tweet and 73% correct for positive tweet over total predicted sentiment. Recall score of negative tweet is 0.72 and positive tweet is 0.76. It means the model is 72% correct for negative tweet and 76% coreect for positive tweet over the total correct predicted sentiment. F1-score is 0.74 for negative tweet and 0.75 for positive tweet. There are not significant difference between negative and positive tweet result for the model. As a result, the accuracy of the model for all tweet is 74%.